In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
FILE_PATH = '/content/drive/MyDrive/Ktra KHTDL/data/Wine.csv'

In [ ]:
df = pd.read_csv(FILE_PATH)
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,class
0,11.6,0.580,0.66,2.20,0.074,10,47,1.00080,3.25,0.57,9.0,Loai 2
1,10.4,0.610,0.49,2.10,0.200,5,16,0.99940,3.16,0.63,8.4,Loai 2
2,7.4,1.185,0.00,4.25,0.097,5,14,0.99660,3.63,0.54,10.7,Loai 2
3,10.4,0.440,0.42,1.50,0.145,34,48,0.99832,3.38,0.86,9.9,Loai 3
4,8.3,1.020,0.02,3.40,0.084,6,11,0.99892,3.48,0.49,11.0,Loai 2


In [ ]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [ ]:
y.unique()

array(['Loai 2', 'Loai 3', 'Loai 4', 'Loai 1', 'Loai 5'], dtype=object)

# **Mã hóa nhãn**

In [ ]:
Le = LabelEncoder()
y = Le.fit_transform(y)
y

array([1, 1, 1, ..., 2, 1, 2])

In [ ]:
np.unique(y)

array([0, 1, 2, 3, 4])

In [ ]:
X = X.to_numpy()
X

array([[11.6  ,  0.58 ,  0.66 , ...,  3.25 ,  0.57 ,  9.   ],
       [10.4  ,  0.61 ,  0.49 , ...,  3.16 ,  0.63 ,  8.4  ],
       [ 7.4  ,  1.185,  0.   , ...,  3.63 ,  0.54 , 10.7  ],
       ...,
       [ 7.2  ,  0.33 ,  0.33 , ...,  3.23 ,  1.1  , 10.   ],
       [ 7.2  ,  0.38 ,  0.31 , ...,  3.23 ,  0.76 , 11.3  ],
       [ 7.4  ,  0.36 ,  0.3  , ...,  3.24 ,  0.7  , 11.4  ]])

# **Câu 1: Chuẩn hóa trong đoạn [a,b]**

In [ ]:
def data_normalization(x,a,b):
  return (b-a)*(x-np.min(x))/(np.max(x)-np.min(x))+a

In [ ]:
a = 0
b = 10

In [ ]:
X = np.apply_along_axis(data_normalization,arr=X, axis = 0, a = a, b= b)

In [ ]:
#In ra X sau khi đã chuẩn hóa dữ liệu
print(X)

[[6.19469027 3.15068493 6.6        ... 4.01574803 1.43712575 0.92307692]
 [5.13274336 3.35616438 4.9        ... 3.30708661 1.79640719 0.        ]
 [2.47787611 7.29452055 0.         ... 7.00787402 1.25748503 3.53846154]
 ...
 [2.30088496 1.43835616 3.3        ... 3.85826772 4.61077844 2.46153846]
 [2.30088496 1.78082192 3.1        ... 3.85826772 2.5748503  4.46153846]
 [2.47787611 1.64383562 3.         ... 3.93700787 2.21556886 4.61538462]]


**Câu 2: Sử dụng thuật toán GaussionNB để phân lớp cho mẫu DL**

In [ ]:
class GaussianNavieBayes:
  def fit(self,X_train, y_train):
    self.X_train = X_train
    self.y_train = y_train
    # Lấy ra các nhãn đơn nhất và số lần xuất hiện của chúng
    self.Labels_unique,count = np.unique(self.y_train,return_counts=True)
    # Tính xác xuất xảy ra của các nhãn
    self.P_labels = count/y_train.size
    # lấy log(probability)
    self.P_labels = np.log(self.P_labels)

    # Mảng mean của feature tương ứng với label = i xảy ra
    self.matrix_mean = np.zeros((self.Labels_unique.size,X_train.shape[1]))
    # Mảng mean của feature tương ứng với label = i xảy ra
    self.matrix_var = np.zeros((self.Labels_unique.size,X_train.shape[1]))

    for i in range(self.Labels_unique.size):
      data_label_i = self.X_train[self.y_train==self.Labels_unique[i]]
      self.matrix_mean[i] = np.mean(data_label_i,axis=0)
      self.matrix_var[i] = np.var(data_label_i,axis=0) + 1e-6


  def predict(self,X_test):
    if X_test.ndim == 1:
      raise ValueError("X_test must be 2D")
    else:
      self.X_test = X_test
      # Xác suất của nhãn i xảy ra khi X_test xảy ra
      self.P_labels_X_test = np.zeros(self.P_labels.shape)
      self.P_labels_X_test+=self.P_labels
      for i in range(self.Labels_unique.size):
        for j in range(self.X_test.shape[1]):
          # Xác suất của đặc trưng i xảy ra khi
          P_featrurej_labeli = np.log(1/np.sqrt(2*np.pi*self.matrix_var[i,j])) -(self.X_test[0,j]-self.matrix_mean[i,j])**2/(2*self.matrix_var[i,j])
          self.P_labels_X_test[i]+= P_featrurej_labeli
      # Tìm giá trị log lớn nhất
      # Kỹ thuật Log sum exp trick -> tránh đưa exp(-1000) về 0
      c = self.P_labels_X_test.max()
      self.P_labels_X_test = np.exp(self.P_labels_X_test-c)
      self.P_labels_X_test_normalization = self.P_labels_X_test/self.P_labels_X_test.sum()
      return self.Labels_unique[np.argmax(self.P_labels_X_test_normalization)]

In [ ]:
X_test = np.array([11.2, 0.5, 0.7, 2.5, 0.07, 12, 50, 1.008, 4.25, 0.57, 10])

In [ ]:
# Kiểm tra shape của data
X.shape

(1599, 11)

In [ ]:
y.shape

(1599,)

In [ ]:
X_test.shape

(11,)

In [ ]:
X_test = X_test.reshape(1,-1)

In [ ]:
clf = GaussianNavieBayes()

In [ ]:
clf.fit(X,y)

In [ ]:
clf.predict(X_test)

np.int64(0)

In [ ]:
# Lấy ra xác suất của các nhãn
clf.P_labels_X_test*100

array([1.00000000e+002, 1.83023072e-019, 1.30081256e-027, 6.87282232e-097,
       8.17220241e-280])

# So sánh với kết quả với kết quả mô hình Gaussian NavieBayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
clf = GaussianNB()

In [ ]:
clf.fit(X,y)

GaussianNB()

In [ ]:
clf.predict(X_test.reshape(1,-1))

array([0])

In [ ]:
clf.predict_proba(X_test)

array([[1.00000000e+000, 1.83019665e-021, 1.30074868e-029,
        6.87080951e-099, 8.16199117e-282]])

# **Câu 4: Dùng thuật toán K_mean để phân cụm DL trên**

In [ ]:
class K_means:
  def __init__(self,n_cluster = 8,max_iter = 100):
    self.n_cluster = n_cluster
    self.max_iter = max_iter
  def fit(self,X):
    if X.ndim ==1:
      raise ValueError("X must be 2D")
    else:
      self.X = X
      self.new_centroids = X[np.random.choice(np.arange(X.shape[0]),self.n_cluster)] # Tạo ra K tâm cụm khởi tạo ngẫu ngiên
      self.old_centroids = np.zeros_like(self.new_centroids)
      self.matrix_distance = np.stack(((np.ones((X.shape[0],1)),)*self.n_cluster),axis=2)
      iter = 0
      while not np.allclose(self.new_centroids,self.old_centroids) and iter < self.max_iter:
        self.old_centroids = np.copy(self.new_centroids)
        for i in range(self.n_cluster):
          self.matrix_distance[:,:,i] = np.sqrt(np.sum((self.X-self.new_centroids[i])**2,axis=1)).reshape(-1,1) # Tính ma trận khoảng cách từ 1 điểm đến tất cả các điểm
        self.idx_matrix = np.argmin(self.matrix_distance,axis=2).flatten()
        self.new_centroids = np.array([np.mean(self.X[self.idx_matrix == i],axis=0) for i in range(self.n_cluster)])
        iter += 1
      self.Labels = self.idx_matrix.reshape(-1,1)
      self.last_centroids = self.new_centroids
  def get_centroids(self):
    return self.last_centroids
  def get_label(self):
    return self.Labels
  def predict(self,X_test):
    if X_test.ndim ==1:
      raise ValueError("X must be 2D")
    else:
      self.X_test = X_test
      self.matrix_distance = np.stack(((np.ones((X_test.shape[0],1)),)*self.n_cluster),axis=2)
      for i in range(self.n_cluster):
        self.matrix_distance[:,:,i] = np.sqrt(np.sum((self.X_test-self.last_centroids[i])**2,axis=1)).reshape(-1,1)
      self.pred = np.argmin(self.matrix_distance,axis=2)
      return self.pred

In [ ]:
Kmean = K_means(n_cluster=4)

In [ ]:
Kmean.fit(X)

In [ ]:
Kmean.get_label()

array([[1],
       [1],
       [0],
       ...,
       [1],
       [2],
       [2]])

In [ ]:
Kmean.get_centroids()

array([[2.56575391, 3.57730283, 1.08272251, 0.92190825, 1.21016121,
        1.59550672, 1.04668874, 4.81034717, 4.95238488, 1.60736119,
        2.37548664],
       [5.21559509, 1.9893591 , 5.02933673, 1.22475189, 1.50199312,
        1.32006324, 0.83904233, 5.91924766, 3.47762333, 2.4051387 ,
        3.34484563],
       [2.21687395, 2.33779156, 2.39222973, 0.94779711, 0.94256193,
        2.21640655, 1.20953109, 3.14122217, 5.25936369, 2.06971193,
        5.35169785],
       [3.23244489, 2.7954527 , 3.05591716, 1.49681851, 1.34123934,
        3.73489457, 2.95816171, 5.42013138, 4.23938872, 1.96896149,
        2.13048096]])